In [1]:
import requests
from dotenv import load_dotenv
import os
import csv
from datetime import datetime, timedelta
import time

In [2]:
load_dotenv()
API_KEY = os.environ.get("OPEN_API")

In [3]:
# Coordinates for Karachi
LAT = 24.8608
LON = 67.0104

# Output CSV file name
CSV_FILENAME = "historical_aqi_karachi.csv"

In [4]:
# --- Date and Time Calculation ---
# Set the end date to today and start date to 6 months (180 days) ago
end_date = datetime.now()
start_date = end_date - timedelta(days=180)

# Convert dates to Unix timestamps for the API
start_timestamp = int(start_date.timestamp())
end_timestamp = int(end_date.timestamp())

In [5]:
print(f"Fetching data from {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")

Fetching data from 2025-04-19 to 2025-10-16


In [6]:
header = [
    'readable_timestamp', 
    'unix_timestamp', 
    'aqi', 
    'co', 
    'no', 
    'no2', 
    'o3', 
    'so2', 
    'pm2_5', 
    'pm10', 
    'nh3'
]

In [7]:
# Open the CSV file in write mode
with open(CSV_FILENAME, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(header) # Write the header first

    # Make the API call to get the entire history in one go
    # NOTE: If this fails for very long ranges, you may need to loop day-by-day
    url = f"http://api.openweathermap.org/data/2.5/air_pollution/history?lat={LAT}&lon={LON}&start={start_timestamp}&end={end_timestamp}&appid={API_KEY}"
    
    print("Making API call for the entire 6-month period...")
    
    try:
        response = requests.get(url)
        # Raise an exception for bad status codes (4xx or 5xx)
        response.raise_for_status() 
        
        data = response.json()
        
        # Check if the 'list' of historical data exists
        if 'list' in data:
            for record in data['list']:
                # Convert unix timestamp to a human-readable string
                readable_time = datetime.fromtimestamp(record['dt']).strftime('%Y-%m-%d %H:%M:%S')
                
                # Extract data for the current row
                row = [
                    readable_time,
                    record['dt'],
                    record['main']['aqi'],
                    record['components']['co'],
                    record['components']['no'],
                    record['components']['no2'],
                    record['components']['o3'],
                    record['components']['so2'],
                    record['components']['pm2_5'],
                    record['components']['pm10'],
                    record['components']['nh3']
                ]
                # Write the row to the CSV file
                writer.writerow(row)
            
            print(f"Successfully processed {len(data['list'])} hourly records.")

    except requests.exceptions.HTTPError as err:
        print(f"HTTP Error: {err}")
        print("Please ensure your API key is valid and your plan supports historical data.")
    except Exception as err:
        print(f"An unexpected error occurred: {err}")

print(f"\nProcess complete. Data saved to {CSV_FILENAME}")

Making API call for the entire 6-month period...
Successfully processed 4314 hourly records.

Process complete. Data saved to historical_aqi_karachi.csv
